# The Impact of Litigations on GHG Emissions
* "litigation" from [Graham Institute](https://climate-laws.org/legislation_and_policies), takes the value of 1 when the first case related to environmental laws was solved in a given year, independently of the result
* "rule of law" from World Bank
* Sample excludes the US, and includes all countries for which there has been at least a case with a resolution: 23 countries, mostly HI and MI countries

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('litigation.csv')

In [3]:
df['id'] = df.groupby('country').ngroup()

In [4]:
import numpy as np
df['lnghg'] = np.log(df['ghg'])
df['lngdp'] = np.log(df['gdp'])
df['lngdp2'] = df['lngdp']**2
df['lnpopulation'] = np.log(df['population'])
df['lnlaw'] = np.log(df['ruleoflaw'])

In [5]:
df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'litigation']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
lnghg,928.0,18.488506,1.495460,14.414347,17.642847,18.265493,19.832111,21.939658
lngdp,905.0,9.986052,0.975039,7.388111,9.420352,10.322941,10.676965,11.700631
lngdp2,905.0,100.670888,18.650197,54.584184,88.743036,106.563113,113.997577,136.904766
lnpopulation,928.0,16.523040,1.576204,12.852783,15.445323,16.385909,17.654756,21.025326
lnlaw,640.0,4.183953,0.470828,2.664199,3.990224,4.367480,4.552512,4.605170
litigation,928.0,0.217672,0.412886,0.000000,0.000000,0.000000,0.000000,1.000000


### Table 3

In [6]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw litigation , robust

import statsmodels.api as sm

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation']].dropna()
X = data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'litigation']]
X = sm.add_constant(X)
y = data['lnghg']
model = sm.OLS(y, X).fit(cov_type='HC3')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lnghg   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.921
Method:                 Least Squares   F-statistic:                     1892.
Date:                Mon, 11 Mar 2024   Prob (F-statistic):               0.00
Time:                        15:39:47   Log-Likelihood:                -346.41
No. Observations:                 639   AIC:                             704.8
Df Residuals:                     633   BIC:                             731.6
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.2282      1.462     -9.730   

In [7]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw litigation i.year, fe robust

from linearmodels.panel import PanelOLS

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation', 'year', 'id']].dropna()
data['cons'] = 1
panel_data = data.set_index(['id', 'year'])

y = panel_data['lnghg']
X = panel_data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'litigation', 'cons']]

model = PanelOLS(y, X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='kernel', kernel='bartlett')
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lnghg   R-squared:                        0.7140
Estimator:                   PanelOLS   R-squared (Between):              0.6795
No. Observations:                 639   R-squared (Within):              -0.2175
Date:                Mon, Mar 11 2024   R-squared (Overall):              0.6722
Time:                        15:39:47   Log-likelihood                    823.42
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      291.14
Entities:                          32   P-value                           0.0000
Avg Obs:                       19.969   Distribution:                   F(5,583)
Min Obs:                       19.000                                           
Max Obs:                       20.000   F-statistic (robust):             1559.3
                            

In [8]:
hi_countries = ['BEL', 'AUS', 'CAN', 'CHE', 'DEU', 'ESP', 'EST', 'FIN', 
                'FRA', 'GBR', 'GRC', 'HRV', 'ITA', 'LTU', 'LUX', 'NLD', 
                'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SWE', 'ROU']
df['HI'] = 0
df.loc[df['country'].isin(hi_countries), 'HI'] = 1

In [9]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation, robust

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation', 'year', 'id', 'HI']].dropna()
data['HITRUE_litigation'] = data['HI'] * data['litigation']
data['HIFALSE_litigation'] = (1-data['HI']) * data['litigation']

X = data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'HITRUE_litigation', 'HIFALSE_litigation']]
X = sm.add_constant(X)
y = data['lnghg']

model = sm.OLS(y, X).fit(cov_type='HC3')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lnghg   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     1607.
Date:                Mon, 11 Mar 2024   Prob (F-statistic):               0.00
Time:                        15:39:47   Log-Likelihood:                -344.84
No. Observations:                 639   AIC:                             703.7
Df Residuals:                     632   BIC:                             734.9
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -13.9922      1

In [10]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation i.year, fe robust

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation', 'year', 'id', 'HI']].dropna()
data['cons'] = 1
data['HITRUE_litigation'] = data['HI'] * data['litigation']
data['HIFALSE_litigation'] = (1-data['HI']) * data['litigation']
panel_data = data.set_index(['id', 'year'])

y = panel_data['lnghg']
X = panel_data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'HITRUE_litigation', 'HIFALSE_litigation', 'cons']]

model = PanelOLS(y, X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='kernel', kernel='bartlett')
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lnghg   R-squared:                        0.7152
Estimator:                   PanelOLS   R-squared (Between):              0.6746
No. Observations:                 639   R-squared (Within):              -0.2016
Date:                Mon, Mar 11 2024   R-squared (Overall):              0.6675
Time:                        15:39:47   Log-likelihood                    824.70
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      243.57
Entities:                          32   P-value                           0.0000
Avg Obs:                       19.969   Distribution:                   F(6,582)
Min Obs:                       19.000                                           
Max Obs:                       20.000   F-statistic (robust):             1346.9
                            